# 0.0 Imports

In [40]:
import pandas            as pd
import numpy             as np
import seaborn           as sns
import matplotlib.pyplot as plt 

from scipy.stats         import shapiro, ttest_ind
from statsmodels.stats   import power, weightstats

## 0.1 - Carregamento dos Dados

In [2]:
df_raw = pd.read_csv('../data/raw/ab_testing.csv')

In [3]:
df_raw.head()

,uid,country,gender,spent,purchases,date,group,device
0,11115722,MEX,F,1595,5,2016-03-08,GRP B,I
1,11122053,USA,M,498,2,2017-07-14,GRP B,I
2,11128688,USA,F,2394,6,2017-09-17,GRP A,I
3,11130578,USA,F,1197,3,2017-11-30,GRP A,I
4,11130759,ESP,M,1297,3,2018-01-10,GRP B,A


In [6]:
# Países
df_raw['country'].unique().tolist()

,uid,country,gender,spent,purchases,date,group,device
0,11115722,MEX,F,1595,5,2016-03-08,GRP B,I
33,11174686,MEX,F,3293,7,2017-10-02,GRP A,I
67,11229291,MEX,M,1198,2,2016-12-23,GRP B,I
81,11253355,MEX,M,598,2,2017-09-09,GRP A,I
98,11286996,MEX,M,3889,11,2016-03-06,GRP B,I
...,...,...,...,...,...,...,...,...
45818,99875377,MEX,F,1897,3,2016-03-23,GRP A,I
45833,99897088,MEX,F,3391,9,2016-12-04,GRP B,I
45840,99913807,MEX,F,3892,8,2015-10-09,GRP B,I
45851,99939956,MEX,F,598,2,2018-01-18,GRP B,I


## 0.2 - Helper Functions  

In [7]:
def sample_size_site_by_country(country, percentual=0.08): 
    
    # DataFrame com os Dados do País
    df_country = df_raw[df_raw['country'] == country].copy()
    
    # DataFrame com os dado do site do país.
    df_country_site = df_country[df_country['device'] == 'I']
    
    # -------------------------- Determinação do Tamanho da Amostra para cada país ------------------------------

    # Nível de confiança
    nivel_confianca = 0.95

    # Nivel de significância
    nivel_significancia = 1 - nivel_confianca 

    # Poder estatístico
    power_stats = 0.8

    # ----------- Tamanho de Efeito ---------------

    # média do grupo de controle
    m1 = np.mean(df_country_site['spent'].values)

    # Média esperada do grupo de tratamento
    m2 = m1*(1+percentual) # percentual default = 0.08

    # Tamanho do Efeito. (Cohen'd)
    effect_size = (m2 - m1)/np.std(df_country_site['spent'].values)

    # Tamanho da Amostra
    tamanho_amostra = power.zt_ind_solve_power(effect_size=effect_size, alpha=nivel_significancia, power=0.8)
    
    
    print('================================================================================')
    print('Lift Esperado: {}'.format(percentual))
    print('País Analisado: {}'.format(country))
    print('Tamanho da Amostra Cada Grupo: {}'.format(int(tamanho_amostra)))
    print('Quantidade de Dados: {}.'.format(df_country_site.shape[0]))
    
    if df_country_site.shape[0] < tamanho_amostra*2:
        print('Falta dados para realizar a amostra. Logo, o teste é inviável.')
        return None
    else:
        print('Dados suficiente para realizar a amostra. Seguir com a validação da hipótese.')
        return int(tamanho_amostra)
    print('=================================================================================')
    print('\n')
    
    
# ---------------------------------------------------------------    
    
    
def sample_size_app_by_country(country, percentual=0.08):
    
    # Variável tem que ser uma string. 
    
    # DataFrame com os Dados do País
    df_country = df_raw[df_raw['country'] == country].copy()
    
    # DataFrame com os dado do site do país.
    df_country_app = df_country[df_country['device'] == 'A']
    
    # -------------------------- Determinação do Tamanho da Amostra para cada país ------------------------------

    # Nível de confiança
    nivel_confianca = 0.95

    # Nivel de significância
    nivel_significancia = 1 - nivel_confianca 

    # Poder estatístico
    power_stats = 0.8

    # ----------- Tamanho de Efeito ---------------

    # média do grupo de controle
    m1 = np.mean(df_country_app['spent'].values)

    # Média esperada do grupo de tratamento
    m2 = m1*(1+percentual)

    # Tamanho do Efeito.
    effect_size = (m2 - m1)/np.std(df_country_app['spent'].values)

    # Tamanho da Amostra
    tamanho_amostra = power.zt_ind_solve_power(effect_size=effect_size, alpha=nivel_significancia, power=0.8)
    
    print('================================================================================')
    print('Lift Esperado: {}'.format(percentual))
    print('País Analisado: {}'.format(country))
    print('Tamanho da Amostra Cada Grupo: {}'.format(int(tamanho_amostra)))
    print('Quantidade de Dados: {}.'.format(df_country_app.shape[0]))
    
    if df_country_app.shape[0] < tamanho_amostra*2:
        print('Falta dados para realizar a amostra. Logo, o teste é inviável.')
        return None
    else:
        print('Dados suficiente para realizar a amostra. Seguir com a validação da hipótese.')
        return int(tamanho_amostra)
    print('=================================================================================')
    print('\n')

# 1.0 - Design de Experimento

## 1.1 - Escolha do Método

**Qual o método que será utilizado?** 
    
    - Será utilizado o teste de hipótese.
    - Usaremos o teste A/B para realizar esse procedimento.
    

## 1.2 - Formulação da Hipóteses

**Observação:** No nosso dataframe, nós temos a exibição dos formulários de preenchimento manual e automático em alguns países que a Eletronic HOuse atende. 

Dessa forma, para realizar o nosso experimento de forma mais eficaz, o teste será realizado por cada país e sobre os dados do site e do app separadamente. 

Logo, as hipóteses formuladas abaixo vão valer para cada país.

### 1.2.1 - Compras realizadas pelo site

**Métrica:** A métrica será a média de quantidade gasta no site.
    
**Hipótese Nula:** O preechimento automático e o preenchimento manual possuem a mesma média de quantidade gasta pelo site.

**Hipótese Alternativa:** O preechimento automático e o preenchimento manual possuem média de compras finalizadas diferentes pelo site.

### 1.2.2 - Compras realizadas pelo app

**Métrica:** A métrica será a média quantidade gasta no app.
    
**Hipótese Nula:** O preechimento automático e o preenchimento manual possuem a mesma média de quantidade gasta pelo app.

**Hipótese Alternativa:** O preechimento automático e o preenchimento manual possuem média de quantidades gastas diferentes pelo app.


## 1.3 - Parâmetros para Determinar o Tamanho da Amostra

### 1.3.1 - Tamanho da Amostra para Compras Realizadas no Site

#### México

In [8]:
# tamanho das amostra para compras realizadas no site no México
sample_size_site_mex = sample_size_site_by_country('MEX')

Lift Esperado: 0.08
País Analisado: MEX
Tamanho da Amostra Cada Grupo: 1243
Quantidade de Dados: 2790.
Dados suficiente para realizar a amostra. Seguir com a validação da hipótese.


#### Estados Unidos

In [9]:
# tamanho das amostra para compras realizadas no site no Estados Unidos
sample_size_site_usa = sample_size_site_by_country('USA')

Lift Esperado: 0.08
País Analisado: USA
Tamanho da Amostra Cada Grupo: 1280
Quantidade de Dados: 6957.
Dados suficiente para realizar a amostra. Seguir com a validação da hipótese.


#### Espanha

In [10]:
# tamanho das amostra para compras realizadas no site no Espanha
sample_size_site_esp = sample_size_site_by_country('ESP')

Lift Esperado: 0.08
País Analisado: ESP
Tamanho da Amostra Cada Grupo: 1180
Quantidade de Dados: 994.
Falta dados para realizar a amostra. Logo, o teste é inviável.


#### Grã-Bretanha


In [11]:
# tamanho das amostra para compras realizadas no site no Grã-Bretanha
sample_size_site_grb = sample_size_site_by_country('GBR', 0.08)

Lift Esperado: 0.08
País Analisado: GBR
Tamanho da Amostra Cada Grupo: 1263
Quantidade de Dados: 1375.
Falta dados para realizar a amostra. Logo, o teste é inviável.


#### Turquia

In [12]:
# tamanho das amostra para compras realizadas no site no Turquia
sample_size_site_tur = sample_size_site_by_country('TUR')

Lift Esperado: 0.08
País Analisado: TUR
Tamanho da Amostra Cada Grupo: 1295
Quantidade de Dados: 1815.
Falta dados para realizar a amostra. Logo, o teste é inviável.


#### Alemanha

In [13]:
# tamanho das amostra para compras realizadas no site no Alemanha
sample_size_site_deu = sample_size_site_by_country('DEU')

Lift Esperado: 0.08
País Analisado: DEU
Tamanho da Amostra Cada Grupo: 1339
Quantidade de Dados: 1848.
Falta dados para realizar a amostra. Logo, o teste é inviável.


#### Brasil

In [14]:
# tamanho das amostra para compras realizadas no site no Brasil
sample_size_site_bra = sample_size_site_by_country('BRA')

Lift Esperado: 0.08
País Analisado: BRA
Tamanho da Amostra Cada Grupo: 1312
Quantidade de Dados: 4517.
Dados suficiente para realizar a amostra. Seguir com a validação da hipótese.


#### França

In [15]:
# tamanho das amostra para compras realizadas no site no França
sample_size_site_fra = sample_size_site_by_country('FRA')

Lift Esperado: 0.08
País Analisado: FRA
Tamanho da Amostra Cada Grupo: 1295
Quantidade de Dados: 1374.
Falta dados para realizar a amostra. Logo, o teste é inviável.


#### Austrália

In [16]:
# tamanho das amostra para compras realizadas no site no Australia
sample_size_site_aus = sample_size_site_by_country('AUS')

Lift Esperado: 0.08
País Analisado: AUS
Tamanho da Amostra Cada Grupo: 1221
Quantidade de Dados: 498.
Falta dados para realizar a amostra. Logo, o teste é inviável.


#### Canada

In [17]:
# tamanho das amostra para compras realizadas no site no Canada
sample_size_site_can = sample_size_site_by_country('CAN')

Lift Esperado: 0.08
País Analisado: CAN
Tamanho da Amostra Cada Grupo: 1321
Quantidade de Dados: 733.
Falta dados para realizar a amostra. Logo, o teste é inviável.


### 1.3.2 - Tamanho da Amostra para Compras Realizadas no App

#### México

In [18]:
# tamanho das amostra para compras realizadas no site no México
sample_size_app_mex = sample_size_app_by_country('MEX')

Lift Esperado: 0.08
País Analisado: MEX
Tamanho da Amostra Cada Grupo: 1269
Quantidade de Dados: 2738.
Dados suficiente para realizar a amostra. Seguir com a validação da hipótese.


#### Estados Unidos

In [19]:
# tamanho das amostra para compras realizadas no site no Estados Unidos
sample_size_app_usa = sample_size_app_by_country('USA')

Lift Esperado: 0.08
País Analisado: USA
Tamanho da Amostra Cada Grupo: 1289
Quantidade de Dados: 7041.
Dados suficiente para realizar a amostra. Seguir com a validação da hipótese.


#### Espanha

In [20]:
# tamanho das amostra para compras realizadas no site no Espanha
sample_size_app_esp = sample_size_app_by_country('ESP')

Lift Esperado: 0.08
País Analisado: ESP
Tamanho da Amostra Cada Grupo: 1197
Quantidade de Dados: 940.
Falta dados para realizar a amostra. Logo, o teste é inviável.


#### Grã-Bretanha

In [21]:
# tamanho das amostra para compras realizadas no site no Grã-Bretanha
sample_size_app_gbr = sample_size_app_by_country('GBR')

Lift Esperado: 0.08
País Analisado: GBR
Tamanho da Amostra Cada Grupo: 1270
Quantidade de Dados: 1434.
Falta dados para realizar a amostra. Logo, o teste é inviável.


#### Turquia

In [22]:
# tamanho das amostra para compras realizadas no site no Turquia
sample_size_app_tur = sample_size_app_by_country('TUR')

Lift Esperado: 0.08
País Analisado: TUR
Tamanho da Amostra Cada Grupo: 1329
Quantidade de Dados: 1722.
Falta dados para realizar a amostra. Logo, o teste é inviável.


#### Alemanha

In [23]:
# tamanho das amostra para compras realizadas no site no Alemanha
sample_size_app_deu = sample_size_app_by_country('DEU')

Lift Esperado: 0.08
País Analisado: DEU
Tamanho da Amostra Cada Grupo: 1232
Quantidade de Dados: 1848.
Falta dados para realizar a amostra. Logo, o teste é inviável.


#### Brasil

In [24]:
# tamanho das amostra para compras realizadas no site no Brasil
sample_size_app_bra = sample_size_app_by_country('BRA')

Lift Esperado: 0.08
País Analisado: BRA
Tamanho da Amostra Cada Grupo: 1338
Quantidade de Dados: 4492.
Dados suficiente para realizar a amostra. Seguir com a validação da hipótese.


#### França

In [25]:
# tamanho das amostra para compras realizadas no site no França
sample_size_app_fra = sample_size_app_by_country('FRA')

Lift Esperado: 0.08
País Analisado: FRA
Tamanho da Amostra Cada Grupo: 1270
Quantidade de Dados: 1468.
Falta dados para realizar a amostra. Logo, o teste é inviável.


#### Austrália

In [26]:
# tamanho das amostra para compras realizadas no site no Australia
sample_size_app_aus = sample_size_app_by_country('AUS')

Lift Esperado: 0.08
País Analisado: AUS
Tamanho da Amostra Cada Grupo: 1348
Quantidade de Dados: 519.
Falta dados para realizar a amostra. Logo, o teste é inviável.


#### Canadá

In [27]:
# tamanho das amostra para compras realizadas no site no Canada
sample_size_app_can = sample_size_site_by_country('CAN')

Lift Esperado: 0.08
País Analisado: CAN
Tamanho da Amostra Cada Grupo: 1321
Quantidade de Dados: 733.
Falta dados para realizar a amostra. Logo, o teste é inviável.


# 2.0 - Prepação e Coleta dos Dados de Amostragem

## 2.1 - Preparação dos Dados

### Verificar a dimensão dos Dados

In [28]:
print('Número de Linhas: {}'.format(df_raw.shape[0]))
print('Número de Colunas: {}'.format(df_raw.shape[1]))

Número de Linhas: 45883
Número de Colunas: 8


### Checagem de NA


In [29]:
df_raw.isna().sum()

uid          0
country      0
gender       0
spent        0
purchases    0
date         0
group        0
device       0
dtype: int64

### Checagem das Duplicidades das Flags

In [30]:
# Checagem da Duplicidade de Flags
df_raw[['uid', 'group']].groupby('uid').count().reset_index().query('group > 2')

,uid,group


### Verificação da Homogeneidade dos Dados dos Sites

In [31]:
(df_raw[['uid', 'group', 'country']][df_raw['device'] == 'I'].groupby(['country', 'group'])
                                                       .agg(count =('uid', 'count') ))

count
country group       
AUS     GRP A    239
        GRP B    259
BRA     GRP A   2310
        GRP B   2207
CAN     GRP A    361
        GRP B    372
DEU     GRP A    931
        GRP B    917
ESP     GRP A    521
        GRP B    473
FRA     GRP A    680
        GRP B    694
GBR     GRP A    684
        GRP B    691
MEX     GRP A   1435
        GRP B   1355
TUR     GRP A    935
        GRP B    880
USA     GRP A   3426
        GRP B   3531

### Verificação da Homogeneidade dos Dados do App

In [32]:
(df_raw[['uid', 'group', 'country']][df_raw['device'] == 'A'].groupby(['country', 'group'])
                                                             .agg(count =('uid', 'count') ))

count
country group       
AUS     GRP A    267
        GRP B    252
BRA     GRP A   2208
        GRP B   2284
CAN     GRP A    337
        GRP B    443
DEU     GRP A    959
        GRP B    889
ESP     GRP A    449
        GRP B    491
FRA     GRP A    753
        GRP B    715
GBR     GRP A    702
        GRP B    732
MEX     GRP A   1451
        GRP B   1287
TUR     GRP A    860
        GRP B    862
USA     GRP A   3501
        GRP B   3540

## 2.2 - Prepação e Coleta dos Dados de Amostragem dos Dados do Site

### Brasil

In [34]:
# Extraindo os dados da amostragem.
df_site_brazil = df_raw[(df_raw['country'] == 'BRA') & (df_raw['device'] == 'I')].copy()

# Realizando amostragem.
df_sample_site_brazil = df_site_brazil.sample(n=sample_size_site_bra)

# Separação de grupo de controle e tratamento
df_control_brazil_site   = df_sample_site_brazil[df_sample_site_brazil['group'] == 'GRP A']['spent']
df_treatment_brazil_site = df_sample_site_brazil[df_sample_site_brazil['group'] == 'GRP B']['spent']

# Cálculo da Taxa de Conversão
avg_spent_control   = np.mean(df_sample_site_brazil[df_sample_site_brazil['group'] == 'GRP A']['spent'].values)
avg_spent_treatment = np.mean(df_sample_site_brazil[df_sample_site_brazil['group'] == 'GRP B']['spent'].values)

# Exibição 
print('Gasto Médio Grupo A: R$ {}'.format(avg_spent_control))
print('Gasto Médio Grupo B: R$ {}'.format(avg_spent_treatment))

Gasto Médio Grupo A: R$ 1891.9804804804805
Gasto Médio Grupo B: R$ 1933.9767801857586


### Mexico

In [35]:
# Extração dos Dados
df_site_mexico = df_raw[(df_raw['country'] == 'MEX') & (df_raw['device'] == 'I')].copy()

# Realização da Amostragem
df_sample_site_mexico = df_site_mexico.sample(n=sample_size_site_mex)

# Separação de grupo de controle e tratamento
df_control_mexico_site   = df_sample_site_mexico[df_sample_site_mexico['group'] == 'GRP A']['spent']
df_treatment_mexico_site = df_sample_site_mexico[df_sample_site_mexico['group'] == 'GRP B']['spent']


# Cálculo da Taxa de Conversão
avg_spent_control   = np.mean(df_sample_site_mexico[df_sample_site_mexico['group'] == 'GRP A']['spent'].values)
avg_spent_treatment = np.mean(df_sample_site_mexico[df_sample_site_mexico['group'] == 'GRP B']['spent'].values)

# Exibição 
print('Gasto Médio Grupo A: R$ {}'.format(avg_spent_control))
print('Gasto Médio Grupo B: R$ {}'.format(avg_spent_treatment))

Gasto Médio Grupo A: R$ 1980.0625
Gasto Médio Grupo B: R$ 1795.318407960199


### USA

In [36]:
# Extração dos Dados
df_site_usa = df_raw[(df_raw['country'] == 'USA') & (df_raw['device'] == 'I')].copy()

# Realização da Amostragem
df_sample_site_usa = df_site_usa.sample(n=sample_size_site_usa)

# Separação de grupo de controle e tratamento
df_control_usa_site   = df_sample_site_usa[df_sample_site_usa['group'] == 'GRP A']['spent']
df_treatment_usa_site = df_sample_site_usa[df_sample_site_usa['group'] == 'GRP B']['spent']

# Cálculo da Taxa de Conversão
avg_spent_control   = np.mean(df_sample_site_usa[df_sample_site_usa['group'] == 'GRP A']['spent'].values)
avg_spent_treatment = np.mean(df_sample_site_usa[df_sample_site_usa['group'] == 'GRP B']['spent'].values)

# Exibição 
print('Gasto Médio Grupo A: R$ {}'.format(avg_spent_control))
print('Gasto Médio Grupo B: R$ {}'.format(avg_spent_treatment))

Gasto Médio Grupo A: R$ 1878.4817749603803
Gasto Médio Grupo B: R$ 1924.9630200308166


## 2.2 - Prepação e Coleta dos Dados de Amostragem dos Dados do App

### Brasil

In [37]:
# Extraindo os dados da amostrage.
df_app_brazil = df_raw[(df_raw['country'] == 'BRA') & (df_raw['device'] == 'A')].copy()

# Realizando amostragem.
df_sample_app_brazil = df_app_brazil.sample(n=sample_size_app_bra)

# Separação de grupo de controle e tratamento
df_control_brazil_app   = df_sample_app_brazil[df_sample_app_brazil['group'] == 'GRP A']['spent']
df_treatment_brazil_app = df_sample_app_brazil[df_sample_app_brazil['group'] == 'GRP B']['spent']


# Cálculo da Taxa de Conversão
avg_spent_control   = np.mean(df_sample_app_brazil[df_sample_app_brazil['group'] == 'GRP A']['spent'].values)
avg_spent_treatment = np.mean(df_sample_app_brazil[df_sample_app_brazil['group'] == 'GRP B']['spent'].values)

# Exibição 
print('Gasto Médio Grupo A: R$ {}'.format(avg_spent_control))
print('Gasto Médio Grupo B: R$ {}'.format(avg_spent_treatment))

Gasto Médio Grupo A: R$ 1831.3476923076923
Gasto Médio Grupo B: R$ 1924.8517441860465


### Mexico

In [38]:
# Extração dos Dados
df_app_mexico = df_raw[(df_raw['country'] == 'MEX') & (df_raw['device'] == 'A')].copy()

# REalização da Amostragem
df_sample_app_mexico = df_app_mexico.sample(n=sample_size_app_mex)

# Separação de grupo de controle e tratamento
df_control_mexico_app   = df_sample_app_mexico[df_sample_app_mexico['group'] == 'GRP A']['spent']
df_treatment_mexico_app = df_sample_app_mexico[df_sample_app_mexico['group'] == 'GRP B']['spent']


# Cálculo da Taxa de Conversão
avg_spent_control   = np.mean(df_sample_app_mexico[df_sample_app_mexico['group'] == 'GRP A']['spent'].values)
avg_spent_treatment = np.mean(df_sample_app_mexico[df_sample_app_mexico['group'] == 'GRP B']['spent'].values)

# Exibição 
print('Gasto Médio Grupo A: R$ {}'.format(avg_spent_control))
print('Gasto Médio Grupo B: R$ {}'.format(avg_spent_treatment))

Gasto Médio Grupo A: R$ 1849.4653179190752
Gasto Médio Grupo B: R$ 1931.2980935875216


### USA


In [39]:
# Extração dos Dados
df_app_usa = df_raw[(df_raw['country'] == 'USA') & (df_raw['device'] == 'A')].copy()

# Realização da Amostragem
df_sample_app_usa = df_app_usa.sample(n=sample_size_app_usa)

# Separação de grupo de controle e tratamento
df_control_usa_app   = df_sample_app_usa[df_sample_app_usa['group'] == 'GRP A']['spent']
df_treatment_usa_app = df_sample_app_usa[df_sample_app_usa['group'] == 'GRP B']['spent']

# Cálculo da Taxa de Conversão
avg_spent_control   = np.mean(df_sample_app_usa[df_sample_app_usa['group'] == 'GRP A']['spent'].values)
avg_spent_treatment = np.mean(df_sample_app_usa[df_sample_app_usa['group'] == 'GRP B']['spent'].values)

# Exibição 
print('Gasto Médio Grupo A: R$ {}'.format(avg_spent_control))
print('Gasto Médio Grupo B: R$ {}'.format(avg_spent_treatment))

Gasto Médio Grupo A: R$ 1808.707592891761
Gasto Médio Grupo B: R$ 1878.4671641791044


# 3.0 - Teste de Hipótese

**Qual o tipo dos Dados?** São dados Contínuos.

**Qual o objetivo do teste?** Encontrar Diferenças.

**Que tipo de diferenças?** Diferenças entre Médias.

**Quantos Grupos?** Dois Grupos.

- Teste de Shapiro sendo Válido: Two Sample Test.
    
- Teste de Shapiro não sendo Válido: Mann-Whitney U

## Brasil

### Site

In [41]:
# Realizar o teste de Shapiro
stats_control, p_valor_control    = shapiro(df_control_brazil_site)
stats_treatment, p_valor_treatment = shapiro(df_treatment_brazil_site)

if p_valor_control < 0.05:
    print('Dados do Grupo Controle são normalmente distribuídos.')
else:
        print('Dados do Grupo Controle não são normalmente distribuídos.')

if p_valor_treatment < 0.05:
    print('Dados do Grupo Tratamento são normalmente distribuídos.')
else:
        print('Dados do Grupo Tratamento não são normalmente distribuídos.')
        
# Teste de Variância
var_control   = np.var(df_control_brazil_site)
var_treatment = np.var(df_treatment_brazil_site)

teste_var = var_control/var_treatment

if teste_var < 4:
    print('Dados possuem Variância igual.')
else:
    print('Dados não possuem variância Igual.')
    

Dados do Grupo Controle são normalmente distribuídos.
Dados do Grupo Tratamento são normalmente distribuídos.
Dados possuem Variância igual.


In [42]:
df_control_brazil_site

18307    2495
18954    2295
14092    1695
4976     2194
2523     2494
         ... 
24144     299
4562     1396
30254     599
10344    2194
21216    1496
Name: spent, Length: 666, dtype: int64

In [44]:
# Realização do Teste
zscore, p_valor = weightstats.ztest(np.array(df_control_brazil_site), 
                                    np.array(df_treatment_brazil_site), 
                                    alternative='two-sided')

print('P-Valor: {}'.format(p_valor))
if p_valor < 0.05:
    print('Rejeitamos a Hipótese nula. Logo, existe um diferença na média de gastos.')
else:
    print('Falhamos em rejeitar a hipótese nula. Concluímos que não há diferença na média de gastos.')

P-Valor: 0.5904843345509335
Falhamos em rejeitar a hipótese nula. Concluímos que não há diferença na média de gastos.


### App

In [46]:
# Realizar o teste de Shapiro
stats_control, p_valor_control     = shapiro(df_control_brazil_app)
stats_treatment, p_valor_treatment = shapiro(df_treatment_brazil_app)

if p_valor_control < 0.05:
    print('Dados do Grupo Controle são normalmente distribuídos.')
else:
        print('Dados do Grupo Controle não são normalmente distribuídos.')

if p_valor_treatment < 0.05:
    print('Dados do Grupo Tratamento são normalmente distribuídos.')
else:
        print('Dados do Grupo Tratamento não são normalmente distribuídos.')
        
# Teste de Variância
var_control   = np.var(df_control_brazil_app)
var_treatment = np.var(df_treatment_brazil_app)

teste_var = var_control/var_treatment

if teste_var < 4:
    print('Dados possuem Variância igual.')
else:
    print('Dados não possuem variância Igual.')
    

Dados do Grupo Controle são normalmente distribuídos.
Dados do Grupo Tratamento são normalmente distribuídos.
Dados possuem Variância igual.


In [47]:
# Realização do Teste
zscore, p_valor = weightstats.ztest(np.array(df_control_brazil_app), 
                                    np.array(df_treatment_brazil_app), 
                                    alternative='two-sided')

print('P-Valor: {}'.format(p_valor))
if p_valor < 0.05:
    print('Rejeitamos a Hipótese nula. Logo, existe um diferença na média de gastos.')
else:
    print('Falhamos em rejeitar a hipótese nula. Concluímos que não há diferença na média de gastos.')

P-Valor: 0.22099079162683677
Falhamos em rejeitar a hipótese nula. Concluímos que não há diferença na média de gastos.


## México

### Site

In [48]:
# Realizar o teste de Shapiro
stats_control, p_valor_control     = shapiro(df_control_mexico_site)
stats_treatment, p_valor_treatment = shapiro(df_treatment_mexico_site)

if p_valor_control < 0.05:
    print('Dados do Grupo Controle são normalmente distribuídos.')
else:
        print('Dados do Grupo Controle não são normalmente distribuídos.')

if p_valor_treatment < 0.05:
    print('Dados do Grupo Tratamento são normalmente distribuídos.')
else:
        print('Dados do Grupo Tratamento não são normalmente distribuídos.')
        
# Teste de Variância
var_control   = np.var(df_control_mexico_site)
var_treatment = np.var(df_treatment_mexico_site)

teste_var = var_control/var_treatment

if teste_var < 4:
    print('Dados possuem Variância igual.')
else:
    print('Dados não possuem variância Igual.')
    

Dados do Grupo Controle são normalmente distribuídos.
Dados do Grupo Tratamento são normalmente distribuídos.
Dados possuem Variância igual.


In [60]:
# Realização do Teste
t_stats, p_valor = weightstats.ztest(np.array(df_control_mexico_site), 
                                     np.array(df_treatment_mexico_site),
                                     alternative='two-sided')

print('P-Valor: {}'.format(p_valor))
if p_valor < 0.05:
    print('Rejeitamos a Hipótese nula. Logo, existe um diferença na média de gastos.')
else:
    print('Falhamos em rejeitar a hipótese nula. Concluímos que não há diferença na média de gastos.')

P-Valor: 0.015882578963210468
Rejeitamos a Hipótese nula. Logo, existe um diferença na média de gastos.


Nessa situação, observamos uma diferença na média de gastos. Como queremos avaliar se o grupo de tratamento possui uma média maior que o grupo tratamento, então nós temos que entrar com o argumento 'smaller' no teste.

In [61]:
# Realização do Teste
t_stats, p_valor = weightstats.ztest(np.array(df_control_mexico_site), 
                                     np.array(df_treatment_mexico_site),
                                     alternative='smaller')

print('P-Valor: {}'.format(p_valor))
if p_valor < 0.05:
    print('Rejeitamos a Hipótese nula. Logo, existe um diferença na média de gastos.')
else:
    print('Falhamos em rejeitar a hipótese nula. Concluímos que não há diferença na média de gastos.')

P-Valor: 0.9920587105183948
Falhamos em rejeitar a hipótese nula. Concluímos que não há diferença na média de gastos.


### App

In [50]:
# Realizar o teste de Shapiro
stats_control, p_valor_control     = shapiro(df_control_mexico_app)
stats_treatment, p_valor_treatment = shapiro(df_treatment_mexico_app)

if p_valor_control < 0.05:
    print('Dados do Grupo Controle são normalmente distribuídos.')
else:
        print('Dados do Grupo Controle não são normalmente distribuídos.')

if p_valor_treatment < 0.05:
    print('Dados do Grupo Tratamento são normalmente distribuídos.')
else:
        print('Dados do Grupo Tratamento não são normalmente distribuídos.')
        
# Teste de Variância
var_control   = np.var(df_control_mexico_app)
var_treatment = np.var(df_treatment_mexico_app)

teste_var = var_control/var_treatment

if teste_var < 4:
    print('Dados possuem Variância igual.')
else:
    print('Dados não possuem variância Igual.')
    

Dados do Grupo Controle são normalmente distribuídos.
Dados do Grupo Tratamento são normalmente distribuídos.
Dados possuem Variância igual.


In [52]:
# Realização do Teste
zscore, p_valor = weightstats.ztest(np.array(df_control_mexico_app), 
                                    np.array(df_treatment_mexico_app), 
                                    alternative='two-sided')

print('P-Valor: {}'.format(p_valor))
if p_valor < 0.05:
    print('Rejeitamos a Hipótese nula. Logo, existe um diferença na média de gastos.')
else:
    print('Falhamos em rejeitar a hipótese nula. Concluímos que não há diferença na média de gastos.')

P-Valor: 0.28413436041439255
Falhamos em rejeitar a hipótese nula. Concluímos que não há diferença na média de gastos.


## USA

### Site

In [53]:
# Realizar o teste de Shapiro
stats_control, p_valor_control     = shapiro(df_control_usa_site)
stats_treatment, p_valor_treatment = shapiro(df_treatment_usa_site)

if p_valor_control < 0.05:
    print('Dados do Grupo Controle são normalmente distribuídos.')
else:
        print('Dados do Grupo Controle não são normalmente distribuídos.')

if p_valor_treatment < 0.05:
    print('Dados do Grupo Tratamento são normalmente distribuídos.')
else:
        print('Dados do Grupo Tratamento não são normalmente distribuídos.')
        
# Teste de Variância
var_control   = np.var(df_control_usa_site)
var_treatment = np.var(df_treatment_usa_site)

teste_var = var_control/var_treatment

if teste_var < 4:
    print('Dados possuem Variância igual.')
else:
    print('Dados não possuem variância Igual.')

Dados do Grupo Controle são normalmente distribuídos.
Dados do Grupo Tratamento são normalmente distribuídos.
Dados possuem Variância igual.


In [54]:
# Realização do Teste
t_stats, p_valor = weightstats.ztest(np.array(df_control_usa_site), 
                                     np.array(df_treatment_usa_site),
                                     alternative='two-sided')

print('P-Valor: {}'.format(p_valor))
if p_valor < 0.05:
    print('Rejeitamos a Hipótese nula. Logo, existe um diferença na média de gastos.')
else:
    print('Falhamos em rejeitar a hipótese nula. Concluímos que não há diferença na média de gastos.')

P-Valor: 0.5490156677233262
Falhamos em rejeitar a hipótese nula. Concluímos que não há diferença na média de gastos.


### App

In [55]:
# Realizar o teste de Shapiro
stats_control, p_valor_control     = shapiro(df_control_usa_app)
stats_treatment, p_valor_treatment = shapiro(df_treatment_usa_app)

if p_valor_control < 0.05:
    print('Dados do Grupo Controle são normalmente distribuídos.')
else:
        print('Dados do Grupo Controle não são normalmente distribuídos.')

if p_valor_treatment < 0.05:
    print('Dados do Grupo Tratamento são normalmente distribuídos.')
else:
        print('Dados do Grupo Tratamento não são normalmente distribuídos.')
        
# Teste de Variância
var_control   = np.var(df_control_usa_app)
var_treatment = np.var(df_treatment_usa_app)

teste_var = var_control/var_treatment

if teste_var < 4:
    print('Dados possuem Variância igual.')
else:
    print('Dados não possuem variância Igual.')
    

Dados do Grupo Controle são normalmente distribuídos.
Dados do Grupo Tratamento são normalmente distribuídos.
Dados possuem Variância igual.


In [56]:
# Realização do Teste
t_stats, p_valor = weightstats.ztest(np.array(df_control_usa_app), 
                                     np.array(df_treatment_usa_app), 
                                     alternative='two-sided')

print('P-Valor: {}'.format(p_valor))
if p_valor < 0.05:
    print('Rejeitamos a Hipótese nula. Logo, existe um diferença na média de gastos.')
else:
    print('Falhamos em rejeitar a hipótese nula. Concluímos que não há diferença na média de gastos.')

P-Valor: 0.3616208095879253
Falhamos em rejeitar a hipótese nula. Concluímos que não há diferença na média de gastos.


# Relatório Final

Não conseguimos seguir com o experimento na Espanha, Grã-Bretanha, Turquia, Alemanha, França, Austrália e Canadá, pois faltam dados para realizar a amostra. Tanto para os dados do site quanto para os dados do App.

Portanto, informamos ao Head de Designers que não há dados suficientes para para realizar a inferência sobre esses países. Logo, a estratégia a ser tomada é:

1. Realizar uma maior coleta de dados. 
    
    - Nessa situação, é importante ter em mente qual vai ser o custo desta coleta de mais amostras para que se tenha uma quantidade mínima para observar se há um lift de 8% na média de gastos dos clientes.


Para os dados do Brasil, México e Estados Unidos, não houve diferença na média de gastos utilizando o formulário automático ou manual, tanto pelo site quanto pelo app.

Nesse contexto, podemos tomar a seguinte atitude com o Head de Designers.

1. Informar ao Head de Designers para solicitar a sua equipe uma melhoria no design do formulário automático.

    - Nessa situação, com uma melhoria no formulário, é esperado que se observe um efeito maior na quantidade média de gastos, isto é, um maior lift esperado em relação ao formulário manual.
    
    
2. Informar ao Head de Designers para solicitar uma nova amostragem.
    
    - Nessa situação, é primordial saber os custos e o tempo necessário para se realizar uma nova coleta de dados, pois com essas informações saberemos se com o lift esperado do formulário automático valerá a pena realizar essa nova coleta de dados.